In [1]:
import requests
import os
import json
import time

bearer_token = os.environ.get("BEARER_TOKEN")
def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "TweetAPIv2Python"
    return r

In [2]:
# get UserId By Name (澤倫斯基userId)

username = "ZelenskyyUa"

def getUserId():
    print("getUserId")
    url = f"https://api.twitter.com/2/users/by?usernames={username}"
    
    response = requests.get(url, auth=bearer_oauth)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
        
    respdata = response.json()
#     print(respdata)
    
    return respdata["data"][0]["id"]

In [3]:
# get Tweets By userId (澤倫斯基推文)

def getUserTweets(userId):
    print("getUserTweets")
    
    url = f"https://api.twitter.com/2/users/{userId}/tweets"
    params = {
        "tweet.fields": "created_at,in_reply_to_user_id,possibly_sensitive,public_metrics,referenced_tweets,source,lang"
    }
    
    response = requests.get(url, auth=bearer_oauth, params=params)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
        
    respdata = response.json()
#     print(respdata["data"][0])
    
    return respdata["data"]

In [4]:
# get Recent By TweetId (推文回覆)

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields

def getTweetRecent(id):
    print("getTweetRecent")
    
    url = "https://api.twitter.com/2/tweets/search/recent"
    params = {
        "query": f"conversation_id:{id}",
        "tweet.fields": "created_at,geo,id,in_reply_to_user_id,lang,referenced_tweets,source,text",
        "expansions": "author_id,referenced_tweets.id",
        "user.fields": "description,created_at"
    }
    
    response = requests.get(url, auth=bearer_oauth, params=params)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
        
    respdata = response.json()
#     print(respdata["data"][0])
    
    return respdata["data"]

In [5]:
# get Retweet By TweetId (推文轉推者)
def getRetweet(id):
    print("getRetweet")
    
    url = f"https://api.twitter.com/2/tweets/{id}/retweeted_by?user.fields=created_at,id,location,name,pinned_tweet_id,public_metrics,url,username"

    response = requests.get(url, auth=bearer_oauth)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)

    respdata = response.json()
    print(len(respdata["data"]))
    lenOfRetweet=len(respdata["data"])
    i=1
    
    for data in respdata["data"]:
        userId = data["id"]
        following=[]
        temp = getFollowing(userId,i,lenOfRetweet)
        following.append(temp)
        data["followings"]=following
        with open('data_RetweetAndFollowing.json', 'w') as f:
            json.dump(respdata, f)
        print(i)
        i+=1
        time.sleep(60)
    
    return respdata["data"]

In [6]:
# get follower By User (使用者跟隨對象)
def getFollowing(id,i,lenOfRetweet):
    print("getFollowing:"+str(i)+'/'+str(lenOfRetweet) +" - "+str(id))
    url = f"https://api.twitter.com/2/users/{id}/following?user.fields=id,location,name,username&max_results=1000"
    
    response = requests.get(url, auth=bearer_oauth)
    
    
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)

    respdata = response.json()
#     print(json.dumps(respdata, indent=4, sort_keys=True))
        
        
    return respdata
    # print(respdata["data"])

In [8]:
def main():
    userId = getUserId()
    tweetsdata = getUserTweets(userId)
    
    print(len(tweetsdata))
    i=1
    for data in tweetsdata:
        now=time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
        if i==3:
            print('i==3 break:'+str(i==3)+'- now:'+now)
            break
            
        print('UserTweets:'+str(i)+'- now:'+now)
        tweetId = data["id"]    
        data["reply"] = getRetweet(tweetId)

        with open('data.json', 'w') as f:
            json.dump(tweetsdata, f)
        print('success ! '+now+' - UserTweets:'+str(i))
        i+=1

if __name__ == "__main__":
    main()

getUserId
getUserTweets
10
UserTweets:1- now:2022-04-06 01:44:53
getRetweet
92
getFollowing:1/92 - 1383560182924333056
1
getFollowing:2/92 - 593391850
2
getFollowing:3/92 - 1191181219087572993
3
getFollowing:4/92 - 1371901854410534923
4
getFollowing:5/92 - 1184355534788415489
5
getFollowing:6/92 - 2741031027
6
getFollowing:7/92 - 878641713807536128
7
getFollowing:8/92 - 1162014228774920192
8
getFollowing:9/92 - 888203517588099072
9
getFollowing:10/92 - 1437411470908735499
10
getFollowing:11/92 - 1445145603995017219
11
getFollowing:12/92 - 260238374
12
getFollowing:13/92 - 889478454290165760
13
getFollowing:14/92 - 199320149
14
getFollowing:15/92 - 1159411999706451968
15
getFollowing:16/92 - 2742918667
16
getFollowing:17/92 - 1162724837464264705
17
getFollowing:18/92 - 938930514274258944
18
getFollowing:19/92 - 2402012870
19
getFollowing:20/92 - 1250553225775890435
20
getFollowing:21/92 - 9745602
21
getFollowing:22/92 - 2150807685
22
getFollowing:23/92 - 1486130327596802048
23
getFollow